In [1]:
import pandas as pd
from influxdb_client import InfluxDBClient
import seaborn as sns
import matplotlib.pyplot as plt

# A few helper functions:
from utils import *

# To illustrate examples
import numpy as np
from scipy.stats import mannwhitneyu, normaltest

url = 'http://192.168.254.89:8086'
token = 'root:root'
org = ''
bucket = 'esp2nred'

with InfluxDBClient(url=url, token=token, org=org) as client:
    query_api = client.query_api()
    #tables = query_api.query('from(bucket: "esp2nred") |> range(start: -7d) |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    #tables = client.query_api().query(query, org=org)
    #for table in tables:
    #    for record in table.records:
    #        print(record)
    # pivot is in the influxdb query
    df = pd.DataFrame(client.query_api().query_data_frame('from(bucket: "esp2nred") |> range(start: -3d) |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")'))
df.head()

ModuleNotFoundError: No module named 'influxdb_client'

In [ ]:
sns.boxplot(data=df, x='location', y='tempf')

In [ ]:
i = sns.catplot(x="location", y="humidityi", data=df, ci='sd', kind="box")

In [ ]:
list(df.location.unique())

In [ ]:
loc = df.loc[(df.location=='4'), :]
print_n_projects(loc, '4')
#print_projects_by(loc, 'device', 'count')

In [ ]:
df.describe() 

In [ ]:
df.groupby('location')['tempf'].describe()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

In [ ]:
dfHist = df[['humidityi', 'tempf']]
#dfHist.head()
#dftest = pd.pivot_table(dfHist, values = 'humidityi', index=['_time'], columns = 'location').reset_index()
#display(dftest)

In [ ]:
fig, axes = plt.subplots(2,5, figsize=(15, 5))
ax = axes.flatten()

for i, col in enumerate(dfHist.columns):
    sns.histplot(dfHist[col], ax=ax[i]) # histogram call
    ax[i].set_title(col)
    # remove scientific notation for both axes
    ax[i].ticklabel_format(style='plain', axis='both')

fig.tight_layout(w_pad=6, h_pad=4) # change padding 
plt.show()
